In [126]:
import re
import pymorphy2
import nltk
import pandas as pd
import numpy as np
import joblib
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.calibration import CalibratedClassifierCV

# Инициализация морфологического анализатора
morph = pymorphy2.MorphAnalyzer()

In [ ]:
# # Загрузка стоп-слов для русского языка (раскомментировать при первом запуске)
# # nltk.download('stopwords')
# russian_stop_words = stopwords.words('russian')

# # Сохранение стоп-слов в файл
# joblib.dump(russian_stop_words, "/Users/tomilovdima/good_bad_news/data/russian_stop_words")

# # Чтение исходных данных
# file_path1 = "/Users/tomilovdima/good_bad_news/data/dataset.txt"
# data = []

# with open(file_path1, 'r', encoding="utf-8") as file:
#     for line in file:
#         line = line.strip()
#         labels_part, text = line.split(" ", 1)
#         labels = labels_part.split(",")
#         text = text.lower()  # Приведение к нижнему регистру
#         data.append({
#             "text": text,
#             "labels": labels
#         })

# # Создание DataFrame
# df = pd.DataFrame(data)

In [ ]:
# # Преобразование multi-label в binary классификацию (TOXIC/NORMAL)
# df["TOXIC"] = df["labels"].apply(lambda x: 1 if "__label__INSULT" in x or "__label__THREAT" in x or "__label__OBSCENITY" in x else 0)
# df["NORMAL"] = df["labels"].apply(lambda x: 1 if "__label__NORMAL" in x else 0)
# df = df.drop(columns="labels")

# # Подготовка признаков и целевой переменной
# X = df["text"]
# y = df["TOXIC"]

In [ ]:
# # Функция предобработки для русского текста (раскомментировать для повторной обработки)
# def preprocess(text):
#     if not isinstance(text, str):
#         return ""
#     text = text.lower()

#     # Удаление символов, кроме букв и пробелов
#     text = re.sub(r'[^а-яё\s]', ' ', text)

#     # Удаление лишних пробелов
#     text = re.sub(r'\s+', ' ', text).strip()

#     # Токенизация
#     tokens = text.split()

#     # Удаление стоп-слов и лемматизация
#     lemmas = []
#     for token in tokens:
#         if token not in russian_stop_words and len(token) > 2:
#             parsed = morph.parse(token)[0]
#             lemmas.append(parsed.normal_form)
    
#     return ' '.join(lemmas)

# # Применение предобработки (раскомментировать для запуска)
# X_processed = [preprocess(text) for text in X]
# df_clean = pd.DataFrame({'text': X_processed, 'label': y})
# df_clean.to_csv('/Users/tomilovdima/good_bad_news/data/X_processed.pkl', index=False, encoding='utf-8')

In [130]:
# Загрузка предобработанных данных
df = pd.read_csv('/Users/tomilovdima/good_bad_news/data/X_processed.pkl')
X_processed = df['text'].values.astype('U')  # Преобразование к Unicode для совместимости
y = df['label'].values
X_processed = np.array(X_processed)

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, 
    test_size=0.2, 
    random_state=1, 
    shuffle=True
)

In [ ]:
# # Пайплайн для GridSearch (раскомментировать для обучения)
# pipeline = Pipeline([
#     ('vectorizer', TfidfVectorizer()),
#     # Оборачиваем LinearSVC для получения вероятностей
#     ('clf', CalibratedClassifierCV(LinearSVC(max_iter=2000, dual='auto')))
# ])

# # Параметры для RandomizedSearch (раскомментировать для обучения)
# param_grid = [
#     {
#         'vectorizer__max_df': [0.7, 0.9],         # Игнорировать слишком частые слова
#         'vectorizer__min_df': [2, 5],             # Игнорировать слишком редкие слова
#         'vectorizer__use_idf': [True, False],     # Взвешивание IDF
#         'vectorizer__max_features': [10000, 20000, 30000],
#         'clf__estimator__C': [0.1, 1, 10, 100],   # Сила регуляризации
#         'clf__method': ['sigmoid', 'isotonic']    # Способы калибровки
#     },
# ]

# # Создание RandomizedSearch (раскомментировать для обучения)
# random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=20, cv=3)
# random_search.fit(X_train, y_train)
# best_model = random_search.best_estimator_
# joblib.dump(best_model, "/Users/tomilovdima/good_bad_news/artefacts/toxic_model_v1.pkl")

['/Users/tomilovdima/good_bad_news/artefacts/toxic_model_v1.pkl']

In [ ]:
# Загрузка сохраненной модели (раскомментировать для использования)
model = joblib.load("/Users/tomilovdima/good_bad_news/artefacts/toxic_model_v1.pkl")

# Пример предсказания
# new_text = ["Ты очень плохой человек"]
# probs = model.predict_proba(new_text)[:, 1]
# print(f"Вероятность токсичности: {probs[0]:.2f}")

Вероятность токсичности: 0.02


In [ ]:
# Предсказание на тестовой выборке и оценка 
y_pred = model.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Комментарий по метрикам:
# precision для NORMAL = 0.98 - из всех сообщений, которые модель пометила как нормальные, 98% действительно нормальные.
# recall для NORMAL = 0.99 - из всех реально нормальных сообщений модель правильно определила 99%.


Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     40714
           1       0.93      0.89      0.91      8944

    accuracy                           0.97     49658
   macro avg       0.96      0.94      0.95     49658
weighted avg       0.97      0.97      0.97     49658

Confusion Matrix:
 [[40156   558]
 [  977  7967]]
